In [1]:
with open("../input.txt") as f:
    input = [line.strip() for line in f.readlines()]
# print(input)

with open("sample-input.txt") as f:
    sample_input = [line.strip() for line in f.readlines()]
print(f"Input: {sample_input}")

Input: ['FF7FSF7F7F7F7F7F---7', 'L|LJ||||||||||||F--J', 'FL-7LJLJ||||||LJL-77', 'F--JF--7||LJLJ7F7FJ-', 'L---JF-JLJ.||-FJLJJ7', '|F|F-JF---7F7-L7L|7|', '|FFJF7L7F-JF7|JL---7', '7-L-JL7||F7|L7F-7F7|', 'L.L7LFJ|||||FJL7||LJ', 'L7JLJL-JLJLJL--JLJ.L']


In [2]:
import numpy as np

# tile mask
tile_mask = {
    '|' : 0b001,
    '-' : 0b010,
    'L' : 0b011,
    'J' : 0b100,
    '7' : 0b101,
    'F' : 0b110,
    'S' : 0b111,
    '.' : 0b000,
}
def tile_from_mask(mask):
    for k, v in tile_mask.items():
        if v == mask:
            return k
    return '?'

neighbor_masks = {
    tile_mask['|'] : np.array([[0, 1, 0], [0, 0, 0], [0, 1, 0]]), # up and down
    tile_mask['-'] : np.array([[0, 0, 0], [1, 0, 1], [0, 0, 0]]), # left and right
    tile_mask['L'] : np.array([[0, 1, 0], [0, 0, 1], [0, 0, 0]]), # right and up
    tile_mask['J'] : np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]), # left and up
    tile_mask['7'] : np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]]), # left and down
    tile_mask['F'] : np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]), # right and down
    tile_mask['S'] : np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]]), # all
}

In [9]:
src = input.copy()

def get_valid_neighbors(row, col, grid):
    # Get the 3x3 neighborhood
    neighborhood = grid[row-1:row+2, col-1:col+2]
    mask = neighbor_masks[grid[row, col]]
    return np.argwhere((neighborhood * mask) > 0) - 1 + [row, col]

def update_distances(grid, dist_grid, curr_dist):
    rows, cols = np.where(dist_grid == curr_dist)
    for row, col in zip(rows, cols):
        neighbors = get_valid_neighbors(row, col, grid)
        for n_row, n_col in neighbors:
            if np.isnan(dist_grid[n_row, n_col]):
                dist_grid[n_row, n_col] = curr_dist + 1

grid = np.array([list(row) for row in src])
mask_grid = np.vectorize(tile_mask.get)(grid)
padded_mask_grid = np.pad(mask_grid, pad_width=1, mode='constant', constant_values=0)

# Initialize distance grid
dist_grid = np.full_like(padded_mask_grid, np.nan, dtype=float)
start_pos = np.argwhere(padded_mask_grid == tile_mask['S'])[0]
curr_dist = 0
dist_grid[tuple(start_pos)] = curr_dist

# check cardinals from start_pos for matching neighbors THAT RECIPROCATE.
# Specification guarantees we only have to care about this once.
neighbors = get_valid_neighbors(*start_pos, padded_mask_grid)
curr_dist += 1
for n_row, n_col in neighbors:
    if np.all(np.any(get_valid_neighbors(n_row, n_col, padded_mask_grid) == start_pos, axis=0)):
        dist_grid[n_row, n_col] = curr_dist

while np.any(np.isnan(dist_grid)) and np.any(dist_grid == curr_dist):
    update_distances(padded_mask_grid, dist_grid, curr_dist)
    curr_dist += 1

# print(dist_grid)

In [10]:
# Replace the ambiguous S with the correct pipe shape based on the connected pipes, for flood filling the right way
start_neighborhood = dist_grid.copy()[start_pos[0]-1:start_pos[0]+2, start_pos[1]-1:start_pos[1]+2].astype(int)

print("Before:\n", grid[start_pos[0] - 2:start_pos[0] + 1, start_pos[1] - 2:start_pos[1] + 1])

start_neighborhood[start_neighborhood != 1] = 0
for k, v in neighbor_masks.items():
    if np.array_equal(start_neighborhood & v, v):
        start_letter = tile_from_mask(k)
        break
grid[start_pos[0] - 1, start_pos[1] - 1] = start_letter

print("After:\n", grid[start_pos[0] - 2:start_pos[0] + 1, start_pos[1] - 2:start_pos[1] + 1])

Before:
 [['J' '|' '|']
 ['7' 'S' 'J']
 ['J' 'F' '-']]
After:
 [['J' '|' '|']
 ['7' 'L' 'J']
 ['J' 'F' '-']]


In [11]:
def create_expanded_grid(fill_grid):
    rows, cols = fill_grid.shape
    expanded_fill_grid = np.full((rows * 2 - 1, cols), ' ', dtype=str)
    expanded_fill_grid[::2] = fill_grid
    return expanded_fill_grid

def update_vertical_connections(expanded_fill_grid):
    for i in range(1, len(expanded_fill_grid), 2):
        row_above = expanded_fill_grid[i - 1]
        row_below = expanded_fill_grid[i + 1] if i + 1 < len(expanded_fill_grid) else None
        connections = np.isin(row_above, ['F', '7', '|']) & np.isin(row_below, ['L', 'J', '|'])
        expanded_fill_grid[i][connections] = '|'
    return expanded_fill_grid

def expand_horizontal(expanded_fill_grid):
    rows, cols = expanded_fill_grid.shape
    horizontal_expanded_grid = np.full((rows, cols * 2 - 1), ' ', dtype=str)
    horizontal_expanded_grid[:, ::2] = expanded_fill_grid
    return horizontal_expanded_grid

def update_horizontal_connections(horizontal_expanded_grid):
    rows, cols = horizontal_expanded_grid.shape
    for i in range(rows):
        for j in range(1, cols, 2):
            col_left = horizontal_expanded_grid[i, j - 1]
            col_right = horizontal_expanded_grid[i, j + 1] if j + 1 < cols else None
            if col_left in ['F', 'L', '-'] and col_right in ['J', '7', '-']:
                horizontal_expanded_grid[i, j] = '-'
    return horizontal_expanded_grid

def flood_fill(grid, x, y, target, replacement):
    stack = [(x, y)]
    while stack:
        x, y = stack.pop()
        if 0 <= x < grid.shape[0] and 0 <= y < grid.shape[1] and grid[x, y] == target:
            grid[x, y] = replacement
            stack.extend([(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)])

fill_grid = np.where(~np.isnan(dist_grid), np.pad(grid, pad_width=1, mode='constant', constant_values=' '), ' ')

# Stretch vertically and expand vertical connections with pipes
expanded_fill_grid = create_expanded_grid(fill_grid)
expanded_fill_grid = update_vertical_connections(expanded_fill_grid)

# Stretch horizontally and expand horizontal connections with bars
horizontal_expanded_grid = expand_horizontal(expanded_fill_grid)
horizontal_expanded_grid = update_horizontal_connections(horizontal_expanded_grid)

# Wash, fill, cut, and dry
horizontal_expanded_grid[horizontal_expanded_grid == '.'] = ' '
flood_fill(horizontal_expanded_grid, 0, 0, ' ', 'O')
final_grid = horizontal_expanded_grid[::2, ::2][1:-1, 1:-1]

print(f"Enclosed: {np.count_nonzero(final_grid == ' ')}")
print("\n".join([''.join(row) for row in final_grid]))

Enclosed: 595
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF7OF7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF7OOF7FJL7||F7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOF-7||F7|||F-J|LJL-7OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO